# Discussion 03

## Principal Components

Welcome to Discussion 03. In this discussion, we'll gain a deeper understanding of principal components and how they are used for dimensionality reduction.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.cluster

plt.rcParams['figure.figsize'] = (7,7)

Let's start by generating some data:

In [ ]:
C = np.array([
    [3, -2],
    [-2, 3]
])

In [ ]:
np.random.seed(42)
X = np.random.multivariate_normal([0,0], C, 300)

In [ ]:
plt.scatter(*X.T)
plt.gca().set_aspect(1)

**Question 01**. What is the *top* eigenvector of the covariance matrix, $C$? What is the *second* eigenvector?

In [ ]:
# BEGIN SOLUTION
eigvals, eigvecs = np.linalg.eigh(C)
u_1 = eigvecs[:,1]
u_2 = eigvecs[:,0]
# END SOLUTION

**Question 02**. Plot both eigenvectors on top of the data. Using color, distinguish the top eigenvector from the second eigenvector.

In [ ]:
# BEGIN SOLUTION
plt.scatter(*X.T)

# scaling by 4 just to make the lines longer...
plt.plot(*np.column_stack([[0, 0], 4*u_1]), color='red', linewidth=3)
plt.plot(*np.column_stack([[0, 0], 4*u_2]), color='C1', linewidth=3)
plt.gca().set_aspect(1)
# END SOLUTION

**Question 03**. For each data point $\vec x^{(i)}$, project it onto the first eigenvector, $\vec u^{(1)}$ to get a new vector: $(\vec x{(i)} \cdot \vec u{(i)}) \, \vec u{(i)}$. Plot your new points on top of the previous plot.

In [ ]:
# BEGIN SOLUTION

# new coefficients
coeffs = X @ u_1

# we need to "replicate" u_1 for each data point and scale it by these coefficients
# we can replicate with np.tile
many_u_1s = np.tile(u_1, (len(X), 1))

# to scale each of the new u_1s, we can multiply with broadcasting, but we
# need to add an axis to get the dimensions to match
new_X = coeffs[:,np.newaxis] * many_u_1s

# this wasn't part of the question, but nice: it plots a dashed line between the old
# point and the new
for i in range(len(X)):
    plt.plot(*np.vstack((X[i], new_X[i])).T, color='k', alpha=.3, linestyle='dashed')

# the original plot from above
plt.scatter(*X.T)
plt.plot(*np.column_stack([[0, 0], 4*u_1]), color='red', linewidth=3)
plt.plot(*np.column_stack([[0, 0], 4*u_2]), color='C1', linewidth=3)
plt.gca().set_aspect(1)

plt.scatter(*new_X.T, color='C2')
# END SOLUTION

**Question 04**. Now let's create a new dataset, $Z$, in the following way. Given a vector $\vec x = (x_1, x_2)^T$, we produce a new representation $\vec z = (z_1, z_2)^T$, where $z_1 = \vec x \cdot \vec u^{(1)}$ and $z_2 = \vec x \cdot \vec u^{(2)}$. Plot $Z$ as a scatter plot.

In [ ]:
# BEGIN SOLUTION
z_1 = X @ u_1
z_2 = X @ u_2
Z = np.column_stack((z_1, z_2))

plt.scatter(*X.T, alpha=.5, color='C2')
plt.scatter(*Z.T)
plt.gca().set_aspect(1)
# END SOLUTION

The following cell will download the MNIST digit dataset:

You should notice that your new scatter plot is a rotated version of the original scatter plot of $X$.

**Question 05**. Compute the covariance matrix for $Z$. What do you notice about the off-diagonal entries? What does this mean, informally?

In [ ]:
# BEGIN SOLUTION
# the off-diagonal entries are close to zero, which means that the new features z_1 and z_2 are uncorrelated
# even more informally, there is little "redundant" information shared between the features
print(np.cov(Z.T))

# END SOLUTION

Now let's try this on a more interesting data set. The cell below will download the MNIST digits.

In [ ]:
%%bash
if [[ ! -e "mnist.npz" ]]; then
    wget 'https://f000.backblazeb2.com/file/jeldridge-data/mnist.npz'
fi

In [ ]:
mnist = np.load('mnist.npz')

Let's use only the "training" data -- there is no training and testing in PCA, it's all just "data".

In [ ]:
X = mnist['train'].T
X.shape

There are 60,000 images in 784 dimensions.

**Question 06**. Compute the covariance matrix. What should be its size? Check to make sure.

In [ ]:
C = np.cov(X.T) # SOLUTION

**Question 07**. Compute the eigenvectors of the covariance matrix.

In [ ]:
eigvals, eigvecs = np.linalg.eigh(C) # SOLUTION

**Question 08**. Each of the eigenvectors is a unit vector -- a direction in 784 dimensions. We can think of each eigenvector as a vector of "mixing coefficients" which creates a particular mixture of the features in the original image (i.e., the pixel intensities). That is, if $\vec u = (u_1, u_2, \ldots, u_{784})^T$, we can think of $u_1$ as being the "mixture coefficient" of pixel 1, $u_2$ is the coefficient of pixel 2, and so forth.

This interpretation allows us to visualize the 784-dimensional eigenvectors as images by reshaping them into 28 x 28 arrays.

Visualize the top 5 eigenvectors of the covariance matrix as images. Do the same, but for the *bottom* five eigenvectors.

In [ ]:
# BEGIN SOLUTION
for u in eigvecs[:,-5:][:,::-1].T:
    plt.figure()
    plt.imshow(u.reshape((28, -1)))
# END SOLUTION

In [ ]:
# BEGIN SOLUTION
for u in eigvecs[:,:5].T:
    plt.figure()
    plt.imshow(u.reshape((28, -1)))
# END SOLUTION